In [48]:
import numpy as np
import math 
import scipy

from itertools import combinations

# Ask for transfer functions what kind of stuff is loaded (line 60)
# Why does the number of Directions depend on the transfer functions ?

def whiteNoise(numberDirections, numberOfSources, hTransfer, numberOfFrequencies):
    ''' White noise localization 
    
     Input:
       - 
       - 
       - 
       - 
    
     Output:
       - 
    '''
    f0 = 0 #starting frequency index
    step = 1 #discretization (for now we want to use the 180 angles so we keep it like that)
    m = 1 #index of used microphone

    
    H_theta0 = np.load('lego1_h_theta_time.npy') #load transfer functions (includes 4 microphones)
    Df = H_theta0.shape[1] #number of directions for a fine discretization
    
    theta = np.random.choice(range(Df), numberOfSources, replace=False) #choose the directions randomly on the fine grid   
        
    H_theta = H_theta0[f0:, ::step, m] #coarse discretization,  model
    [F, D] = H_theta.shape #F: number of frequencies,  D: number of directions
    
    H_theta0_t = np.load('lego1_h_theta_time.npy')[:, :, m]
    H_theta_t = H_theta0_t[:, ::step] #coarse discretization

    runs = 1
    n_samples = 256
    J = numberOfSources
    obs_len = n_samples + H_theta_t.shape[0] - 1#length of the convolution
    print("obs_len is %s" % obs_len)
    SNR = 20 # noise of 20 dB
    Fn = n_samples/2. +1 #number of frequencies in spectrogram (not sure at all)
    
    for rns in range(runs):
        St_all = np.zeros((n_samples, J)) #list of source signals
        for j in range(J):
            St_all[:, j] = np.random.randn(n_samples) #source in time: random gaussian signal 

        theta = np.random.choice(range(Df), J, replace=False) #choose the directions randomly on the fine grid   

        yt = np.zeros((obs_len, )) #recorded time domain signal

        for j in range(J):
            print("St shape is %s" %St_all[:, j].shape)
            print("H_theta0_t shape is %s" % H_theta0_t[:, theta[j]].shape)
            yt += np.convolve(St_all[:, j], H_theta0_t[:, theta[j]]) #source signal convolved with corresponding directional response
        print("yt shape is %s" %yt.shape)

        # Generate noise at required SNR    
        sig_norm = np.linalg.norm(yt)
        noise_t = np.random.randn(obs_len, ) #additive gaussian noise
        noise_norm = sig_norm/(10.**(SNR/20.))
        noise_t = noise_norm*noise_t/np.linalg.norm(noise_t)

        yt += noise_t #noisy signal
        
        print("yt shape is %s" %yt.shape)


        y = stft(yt, Fn)[f0:, :] #spectrogram of recorded signal
        N = y.shape[1] #number of frames

        y_mean = np.mean(np.abs(y)**2, axis=1) #mean power frame
        y_mean = y_mean/np.linalg.norm(y_mean) #normalize the observation

        # Exhaustive search algorithm

        # Initialize variables
        best_ind = np.inf #index corresponding to best direction tuple
        smallest_norm = np.inf #smallest projection error
        best_dir = theta #best direction tuple

        # Search all combinations
        pairs2 = combinations(range(D), J)
        for q2, d2 in enumerate(pairs2): 
            Bj = np.abs(H_theta[:, d2])**2 #vectors in current guess
            Pj = Bj.dot(np.linalg.pinv(Bj)) #projection matrix
            print("eye shape is %s" %np.eye(F).shape)
            print("eye shape is %s" %np.eye(F).shape)
            
            proj_error = np.linalg.norm((np.eye(F) - Pj).dot(y_mean)) #projection error

            if proj_error <= smallest_norm:
                smallest_norm = proj_error
                best_ind = q2
                best_dir = d2
        theta_hat = step*np.array(best_dir) #map coarse index to fine index
        min_err, best_perm = calculate_angle_error(theta, theta_hat, anglesf) #calculate error between chosen and true directions
        conf_matrix[theta, best_perm] += 1
        
        for src_j in range(J): #error per source
            err_per_source[rns, src_j] = np.sum(np.absolute(((best_perm[src_j]-theta[src_j]+180) % 360)-180));
    
    
    return Df, H_theta0.shape, H_theta.shape   


# Taken from https://github.com/swing-research/scatsense/blob/master/core/signal.py
# maybe possible to change it to just have rfft ?
def stft(x,F):
    '''Compute STFT of 1D real signal x
    F: number of frequencies
    '''
    wn = 2*(F-1) #window size in samples
    nhop = int(wn/2) #hop size, 50% overlap
    
    L = len(x) #length of input signal
    zero_pad = nhop #number of zeros to add before and after the signal
    rem = int(L%nhop) #number of samples leftover
    
    if rem>0: # adjust zero padding to have an integer number of windows
        zero_pad = int(wn-rem)#nhop-rem#wn-rem
    x = np.hstack((np.zeros((nhop,)),x,np.zeros((zero_pad,)))) #zero padding at the beginning to avoid boundary effects
    L = len(x) #new length of signal
    
    N = int((L-wn)/nhop + 1) #total number of frames
    X = np.zeros((int(F),N),dtype=np.complex128) #output STFT matrix
     
   
    w = 0.5- 0.5*np.cos(2*np.pi*np.arange(wn)/(wn)) #Hann window
    for i in range(N): #compute windowed fft for every frame
        xf = np.fft.rfft(x[int(i*nhop):int(i*nhop+wn)]*w)
        X[:,i] = xf

    return X 

whiteNoise(2, 1, np.array([[1,2],[5,6]]), 5)

obs_len is 415
St shape is 256
H_theta0_t shape is 160
yt shape is 415
yt shape is 415


ValueError: shapes (160,160) and (129,) not aligned: 160 (dim 1) != 129 (dim 0)